In [2]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [4]:
from datetime import datetime

import ontime as on
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset

from ontime.module.processing.pytorch.utils import create_dataset
from ontime.module.processing.common import split_in_windows, split_inputs_from_targets, timeseries_list_to_numpy
from ontime.module.processing.common import train_test_split, normalize

# PyTorch Dataset

## Create test TimeSeries

In [5]:
v1 = np.arange(0,100)
v2 = np.arange(100,200)
arr = np.column_stack((v1,v2))

start_date = pd.Timestamp('2023-01-01')
time_indices = pd.date_range(start=start_date, periods=len(arr), freq='D')

ts = on.TimeSeries.from_times_and_values(time_indices, arr)

## Create Splits

In [7]:
train, test = train_test_split(ts, test_split=0.3)
train_enc, transformer = normalize(train, return_transformer=True)
test_enc = transformer.transform(test)

## Create DataSets

In [8]:
ds_train = create_dataset(
    train_enc,
    window_length = 10,
    stride_length = 10,
    input_length = 6,      
    target_length = 4
)

ds_test = create_dataset(
    test_enc,
    window_length = 10,
    stride_length = 10,
    input_length = 6,      
    target_length = 4
)

## Check samples encoding

Get a sample

In [44]:
input = ds_train.data_array[0]
target = ds_train.labels_array[0]

Print input

In [46]:
transformer.inverse_transform(on.TimeSeries.from_data(input.T))

<TimeSeries (DataArray) (time: 6, component: 2, sample: 1)>
array([[[  0.],
        [100.]],

       [[  1.],
        [101.]],

       [[  2.],
        [102.]],

       [[  3.],
        [103.]],

       [[  4.],
        [104.]],

       [[  5.],
        [105.]]])
Coordinates:
  * time       (time) int64 0 1 2 3 4 5
  * component  (component) <U1 '0' '1'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

Print target

In [47]:
transformer.inverse_transform(on.TimeSeries.from_data(target.T))

<TimeSeries (DataArray) (time: 4, component: 2, sample: 1)>
array([[[  6.],
        [106.]],

       [[  7.],
        [107.]],

       [[  8.],
        [108.]],

       [[  9.],
        [109.]]])
Coordinates:
  * time       (time) int64 0 1 2 3
  * component  (component) <U1 '0' '1'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None